# Goal of the notebook

In order to apply classical models (such as Black-Scholes model which we will use as a benchmark), we need to create some very basic features (such as time until expiration and risk-free rate). In this notebook, we will do it.

In [15]:
import numpy as np
import pandas as pd

In [16]:
dataset_full = pd.read_csv('Raw_data_02_05.csv')
dataset_full

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (4,5,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility,Expiration date,Company,Date of getting data
0,AAPL240223C00095000,2024-02-15 10:56AM EST,95.0,86.97,87.1,87.6,0.00,-,1,7,177.34%,"February 23, 2024",AAPL,16 February
1,AAPL240223C00100000,2024-02-16 3:20PM EST,100.0,82.27,82.1,82.65,0.06,+0.07%,1,9,173.44%,"February 23, 2024",AAPL,16 February
2,AAPL240223C00110000,2024-01-22 12:07PM EST,110.0,83.50,72.1,72.65,0.00,-,-,1,148.05%,"February 23, 2024",AAPL,16 February
3,AAPL240223C00115000,2024-02-14 3:52PM EST,115.0,66.87,67.1,67.65,-2.18,-3.16%,1,2,135.94%,"February 23, 2024",AAPL,16 February
4,AAPL240223C00120000,2024-02-15 3:57PM EST,120.0,64.40,62.1,62.65,0.00,-,6,3,124.61%,"February 23, 2024",AAPL,16 February
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1223326,INTC261218C00050000,2024-04-12 3:01PM EDT,50.0,5.70,5.55,5.8,-0.80,-12.31%,42,1420,42.66%,"December 18, 2026",INTC,12 April
1223327,INTC261218C00055000,2024-04-12 2:27PM EDT,55.0,4.80,4.7,4.8,-0.60,-11.11%,73,31727,42.29%,"December 18, 2026",INTC,12 April
1223328,INTC261218C00060000,2024-04-12 2:36PM EDT,60.0,3.95,3.8,4.05,-0.57,-12.61%,68,2618,42.29%,"December 18, 2026",INTC,12 April
1223329,INTC261218C00065000,2024-04-12 2:37PM EDT,65.0,3.30,2.52,3.35,-0.47,-12.47%,22,304,41.89%,"December 18, 2026",INTC,12 April


### Subtract expiration date from date of collection

In [17]:
dataset_full[['Exp_date', 'Exp_year']] = dataset_full['Expiration date'].str.split(',', expand=True)
dataset_full[['Exp_month', 'Exp_day']] = dataset_full['Exp_date'].str.split(' ', expand=True)
diction = {"January" : 1, "February" : 2, "March": 3, "April" : 4, 
           "May": 5, 'June': 6, 'July': 7, 'August': 8,
          'September': 9, 'October': 10, 'November': 11, 'December': 12}
dataset_full = dataset_full.replace({"Exp_month": diction})
dataset_full['Get_year'] = 2024
dataset_full[['Get_day', 'Get_month']] = dataset_full['Date of getting data'].str.split(' ', expand=True)
dataset_full = dataset_full.replace({"Get_month": diction})
dataset_full

,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,...,Expiration date,Company,Date of getting data,Exp_date,Exp_year,Exp_month,Exp_day,Get_year,Get_day,Get_month
0,AAPL240223C00095000,2024-02-15 10:56AM EST,95.0,86.97,87.1,87.6,0.00,-,1,7,...,"February 23, 2024",AAPL,16 February,February 23,2024,2,23,2024,16,2
1,AAPL240223C00100000,2024-02-16 3:20PM EST,100.0,82.27,82.1,82.65,0.06,+0.07%,1,9,...,"February 23, 2024",AAPL,16 February,February 23,2024,2,23,2024,16,2
2,AAPL240223C00110000,2024-01-22 12:07PM EST,110.0,83.50,72.1,72.65,0.00,-,-,1,...,"February 23, 2024",AAPL,16 February,February 23,2024,2,23,2024,16,2
3,AAPL240223C00115000,2024-02-14 3:52PM EST,115.0,66.87,67.1,67.65,-2.18,-3.16%,1,2,...,"February 23, 2024",AAPL,16 February,February 23,2024,2,23,2024,16,2
4,AAPL240223C00120000,2024-02-15 3:57PM EST,120.0,64.40,62.1,62.65,0.00,-,6,3,...,"February 23, 2024",AAPL,16 February,February 23,2024,2,23,2024,16,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1223326,INTC261218C00050000,2024-04-12 3:01PM EDT,50.0,5.70,5.55,5.8,-0.80,-12.31%,42,1420,...,"December 18, 2026",INTC,12 April,December 18,2026,12,18,2024,12,4
1223327,INTC261218C00055000,2024-04-12 2:27PM EDT,55.0,4.80,4.7,4.8,-0.60,-11.11%,73,31727,...,"December 18, 2026",INTC,12 April,December 18,2026,12,18,2024,12,4
1223328,INTC261218C00060000,2024-04-12 2:36PM EDT,60.0,3.95,3.8,4.05,-0.57,-12.61%,68,2618,...,"December 18, 2026",INTC,12 April,December 18,2026,12,18,2024,12,4
1223329,INTC261218C00065000,2024-04-12 2:37PM EDT,65.0,3.30,2.52,3.35,-0.47,-12.47%,22,304,...,"December 18, 2026",INTC,12 April,December 18,2026,12,18,2024,12,4


In [18]:
dataset_full['Date_get_data'] = pd.to_datetime(dict(year=dataset_full.Get_year, month=dataset_full.Get_month, day=dataset_full.Get_day))
dataset_full['Date_expiration'] = pd.to_datetime(dict(year=dataset_full.Exp_year, month=dataset_full.Exp_month, day=dataset_full.Exp_day))
dataset_full = dataset_full.drop(['Exp_day', 'Exp_year', 'Exp_month', 'Get_year', 'Get_day', 'Get_month',
                                 'Date of getting data', 'Expiration date',
                                 'Last Trade Date', 'Exp_date'], axis=1)

dataset_full['Time_to_maturity'] = (dataset_full['Date_expiration'] - dataset_full['Date_get_data']).dt.days
dataset_full['Time_to_maturity'] = dataset_full['Time_to_maturity'] / 365 # do as in our source
dataset_full['r'] = 0.04 # assume it to be fixed and to be 4%
dataset_full.shape

(1223331, 15)

In [19]:
dataset_full = dataset_full.loc[dataset_full['Contract Name'].notna()]
dataset_full.shape

(1223326, 15)

In [20]:
dataset_full.head()

,Contract Name,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility,Company,Date_get_data,Date_expiration,Time_to_maturity,r
0,AAPL240223C00095000,95.0,86.97,87.1,87.6,0.00,-,1,7,177.34%,AAPL,2024-02-16,2024-02-23,0.019178,0.04
1,AAPL240223C00100000,100.0,82.27,82.1,82.65,0.06,+0.07%,1,9,173.44%,AAPL,2024-02-16,2024-02-23,0.019178,0.04
2,AAPL240223C00110000,110.0,83.50,72.1,72.65,0.00,-,-,1,148.05%,AAPL,2024-02-16,2024-02-23,0.019178,0.04
3,AAPL240223C00115000,115.0,66.87,67.1,67.65,-2.18,-3.16%,1,2,135.94%,AAPL,2024-02-16,2024-02-23,0.019178,0.04
4,AAPL240223C00120000,120.0,64.40,62.1,62.65,0.00,-,6,3,124.61%,AAPL,2024-02-16,2024-02-23,0.019178,0.04


In [21]:
dataset_full.to_csv('Maturity_data_02_05.csv', index=None)

### Complete